# Табличные данные и взаимодействие с Excel

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Табличные данные и взаимодействие с Excel"
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.org/github/pybokeh/jupyter_notebooks/blob/main/xlwings/Excel_Formatting.ipynb
* https://xlsxwriter.readthedocs.io/
* https://xlsxwriter.readthedocs.io/working_with_conditional_formats.html
* https://xlsxwriter.readthedocs.io/example_conditional_format.html
* https://ru.excelfunctions.eu/
* https://docs.python.org/3/library/csv.html
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot
* https://analysistabs.com/excel-vba/colorindex/

In [1]:
pip install xlsxwriter


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Задачи для совместного разбора

In [2]:
import xlsxwriter
import xlwings as xw
import numpy as np
import pandas as pd
import csv

1. Создать файл `demo.xlsx` при помощи пакета `xlswriter` на основе данных из файла `costs.csv`.

In [3]:
workbook = xlsxwriter.Workbook('demo.xlsx')
worksheet = workbook.add_worksheet('Рецептура')

# Some data we want to write to the worksheet.

row = 0
col = 0

data = []

for line in open('costs.csv', 'r', encoding='UTF-8'):
    data.append(str(line).split())

worksheet.write(row, col, ' ')

col += 1

for item in (data[0])[:5]:
    worksheet.write(row, col, item)
    col += 1

worksheet.write(row, col, f'{data[0][6]} {data[0][7]}')
col += 1
worksheet.write(row, col, f'{data[0][8]} {data[0][9]}')
col += 1

row += 1

for el in data[1:]:
    if row == 4:
        worksheet.write(row, 0, f'{el[0]}')
        worksheet.write_number(row, 1, float(el[1]))
        col = 2
    else: 
        worksheet.write(row, 0, f'{el[0]} {el[1]}')
        col = 1
    for item in el[2:]:
        worksheet.write_number(row, col, float(item))
        col += 1
    row += 1

2. На листе "Рецептура" файла `demo.xlsx` рассчитать себестоимость всех видов продукции. Результаты расчетов сохранить в отдельном столбце.

In [4]:
worksheet.write('I1', 'Суммарно')
for row in range(2,7):
    worksheet.write_formula(f'I{row}', f'=SUM(B{row}:H{row})')

# workbook.close()

3. Выполнить 2 с помощью "протягиваемых" формул.

In [5]:
worksheet.write_formula('I1:I6', '=SUM(B2:H6)')
worksheet.write('I1', 'Суммарно')
# workbook.close()

0

4. Добавить общий заголовок и доработать форматирование новых столбцов

In [6]:
worksheet.set_column('A:A', 20)
worksheet.set_column('B:I', 15)

cell_format = workbook.add_format({'bold': True})
worksheet.set_row(0, None, cell_format)

workbook.close()

## Лабораторная работа 4

1. При помощи пакета `xlsxwriter` создайте рабочую книгу под названием `recipes_model.xlsx`. 
Добавьте лист с названием "Модель". В первой строчке листа разместите следующие названия колонок: `"Сущность", "Атрибут", "Тип данных", "Обязательно к заполнению", "Описание сущности", "Описание атрибута", "Ключ", "Таблица для ссылки", "Поле для ссылки", "SQL"`.

* Не закрывайте рабочую книгу, она потребуется для выполнения следующего задания.

In [7]:
workbook = xlsxwriter.Workbook('recipes_model.xlsx')
worksheet = workbook.add_worksheet('Модель')

headers = ["Сущность", "Атрибут", "Тип данных", "Обязательно к заполнению", "Описание сущности", "Описание атрибута", "Ключ", "Таблица для ссылки", "Поле для ссылки", "SQL"]

for header in headers:
    worksheet.write(0 ,headers.index(header), header)

# workbook.close()

2. При помощи пакета `xlsxwriter` добавьте условное форматирование на лист "Модель":
* если в ячейке столбца `G` указано значение "PK", то сделать заливку ячейки цвета  `#006600`, а цвет шрифта - `#fffff`.
* если в ячейке столбца `G` указано значение "FK", то сделать заливку ячейки цвета  `#cccc00`, а цвет шрифта - `#000000`.

Примените данные правила к диапазону `G2:G50`. Закройте рабочую книгу.

In [8]:
# green = workbook.add_format({'color': 'green'})
# bold_red = workbook.add_format({'bold': True, 'color': 'red'})
# worksheet.write_rich_string('A1', green,
#                             'hello ',
#                             bold_red, 'this is bold_red', # style
#                             ' and default string')

cell_format_PK = workbook.add_format({'font_color': '#fffff', 'bg_color': '#006600'})
cell_format_FK = workbook.add_format({'font_color': '#000000', 'bg_color': '#cccc00'})


worksheet.conditional_format('G2:G50', {'type':'cell',
                                    'criteria':'equal to',
                                    'value':'"PK"',
                                    'format':cell_format_PK})
worksheet.conditional_format('G2:G50', {'type':'cell',
                                    'criteria':'equal to',
                                    'value':'"FK"',
                                    'format':cell_format_FK})

# for i in range (2,51):
#     if worksheet.table.get(f'G{i}', None) == 'PK':
#         worksheet.write(i, 'G', 'PK', cell_format_PK)
#     if worksheet.table.get(f'G{i}', None) == 'FK':
#         worksheet.write(i, 'G', 'FK', cell_format_FK)


workbook.close()

3. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [9]:
import pandas as pd
wb = xw.Book('recipes_model.xlsx')
model = wb.sheets['Модель']

with open('recipes_model.csv', newline='',  encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    # col = 1
    # row = 2
    # for line in reader:
    #     col = 1
    #     for el in line[0].replace('\t', ',').split(','):
    #         if el == '':
    #             model.range(row, col).value = ' '
    #         else:
    #             model.range(row, col).value = el
    #         col += 1
    #     row += 1
    data = pd.DataFrame(reader)

model.range('A2').options(index=False, header=False).value = data


# wb.save()
# wb.close()

4. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [10]:
# from xlwings.ext.sql import sql

# workbook = xw.Book('recipes_model.xlsx')

# t1 = workbook.sheets['Модель']['B1:I18'].value


# workbook.sheets['Модель']['J1'].value= sql("""
# select Атрибут ' ' upper(`Тип данных`)||' '|| 
# (case when Ключ='PK' then 'PRIMARY KEY' when Ключ='FK' then 
# ('REFERENCES' ' ' Таблица для ссылки '(' Поле для ссылки  ')')  else '' end) ||' '
# (case when `Обязательно к заполнению`='Y' and (Ключ<>'PK' or Ключ is null) then 'NOT NULL' else '' end)  
# from a""", t1)
# workbook.sheets['Модель']['J1'].value = ('SQL')

# workbook.close()

5. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [11]:
xw.Range('A1:J1').color = ('#00ccff')
xw.Range('A:J').autofit()
xw.Range('A1:J1').font.bold = True
# xw.Range('A1').wrap_text = True

# wb.save()
# wb.close()

6. В данных, полученных из файла `recipes_model.csv`, имеются ошибки двух видов:

    1\. Для некоторых полей, указанных как первичный ключ, в столбце "Обязательно к заполнению" стоит значение "N";

    2\. Для некоторых полей, указанных как внешний ключ, не указана таблица для ссылки и поле для ссылки.

Получите данные из листа в виде `pd.DataFrame`. Найдите строки на листе, которые содержат ошибки первого или второго рода и сделайте заливку эти строк цвета `(255, 102, 102)`. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [12]:
ERR1 = []
ERR2 = []

counter = 2
for el in data[3]:
    if el == 'N':
        xw.Range(f'D{counter}').color = (255, 102, 102)
        ERR1.append(f'D{counter}')
    counter += 1

counter = 2
for el in data[7]:
    if len(el) == 0:
        xw.Range(f'H{counter}').color = (255, 102, 102)
        ERR2.append(f'H{counter}')
    counter += 1

counter = 2
for el in data[8]:
    if len(el) == 0:
        xw.Range(f'I{counter}').color = (255, 102, 102)
        ERR2.append(f'I{counter}')
    counter += 1

wb.save()
wb.close()

7. При помощи пакета `xlwings` создайте лист с названием `Ошибки заполнения`. 

Поместите в первой строчке этого листа заголовки "№ строки" и "Ошибка". Заполните лист на основе решения задачи 6. В столбце "№ строки" укажите номер строки с листа `Модель`, содержащую ошибку, а в столбце "Ошибка" для ошибок первого рода напишите сообщение "Первичный ключ не может быть NULL", для ошибок второго рода - "Не указано поле для связи по внешнему ключу". Сделайте скриншот состояния листа `Ошибки заполнения` и прикрепите в ячейку ноутбука. 

In [13]:
print(ERR1, ERR2)

['D3', 'D7', 'D8', 'D9', 'D15'] ['H2', 'H3', 'H4', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'I2', 'I3', 'I4', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I13', 'I14', 'I15', 'I16', 'I17', 'I18']


In [14]:
wb = xw.Book('recipes_model.xlsx')
wb.sheets.add('Ошибки заполнения')
err_sh = wb.sheets['Ошибки заполнения']

err_1 = "Первичный ключ не может быть NULL"
err_2 = "Не указано поле для связи по внешнему ключу"

data1 = pd.DataFrame(ERR1, columns=['№ строки'])
data2 = pd.DataFrame(ERR2)

data1['Ошибка'] = err_1
data2['Ошибка'] = err_2

err_sh.range('A1').options(index=False, header=True).value = data1
err_sh.range(f'A{data1.shape[0]+1}').options(index=False, header=False).value = data2
xw.Range('B6').autofit()

# print(data1.shape[0]) #5

wb.save()
wb.close()

8. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [15]:
# wb = xw.Book('recipes_model.xlsx')
# wb.sheets.add('Статистика')
# stats = wb.sheets['Статистика']

# for el in range(data.shape[0]):
#     stats.range(f'A{el + 1}').options(index=False, header=False).value = data.shape[1] - (data == '').sum(axis=1)[el]

# wb.save()
# wb.close()

In [16]:
# worksheet = workbook.get_worksheet_by_name('Статистика')

# chart = workbook.add_chart({'type': 'column'})
# for el in range(data.shape[0]):
#     chart.add_series({'values': f'=A{el + 1}'})

# worksheet

# wb.save()
# wb.close()

In [17]:
workbook = xlsxwriter.Workbook('recipes_model.xlsx')
worksheet = workbook.add_worksheet('Статистика')

data3 = []

for el in range(data.shape[0]):
    data3.append(data.shape[1] - (data == '').sum(axis=1)[el])


worksheet.write_column('A1', data3)

# Create a new chart object.
chart = workbook.add_chart({'type': 'column'})

# Add a series to the chart.
chart.add_series({'values': f'=Статистика!A$1$:A${data.shape[0]}$'})

# Insert the chart into the worksheet.
worksheet.insert_chart('E2', chart)

workbook.close()